In [1]:
!pip uninstall -y numpy -q
!pip install numpy==1.23.5 --force-reinstall --no-cache-dir
!pip install --force-reinstall --no-cache-dir git+https://github.com/NicolasHug/Surprise.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 78.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
bigframes 2.5.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>

  Cloning https://github.com/NicolasHug/Surprise.git to /tmp/pip-req-build-w7ilo3sq
  Running command git clone --filter=blob:none --quiet https://github.com/NicolasHug/Surprise.git /tmp/pip-req-build-w7ilo3sq
  Resolved https://github.com/NicolasHug/Surprise.git to commit 2381fb11d0c4bf917cc4b9126f205d0013649966
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
^C


TUGAS AKHIR PEMBELAJARAN MESIN

disini cbf saya hanya dihitung dari produk unik pak, agar ram saya lebih efisien sehingga tidak terjadi crash.

In [1]:
!pip uninstall -y numpy -q
!pip install numpy==1.23.5 -q
!pip install git+https://github.com/NicolasHug/Surprise.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 57.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
bigframes 2.5.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>

In [2]:
!pip install git+https://github.com/NicolasHug/Surprise.git

  Cloning https://github.com/NicolasHug/Surprise.git to /tmp/pip-req-build-h9qj8ycd
  Running command git clone --filter=blob:none --quiet https://github.com/NicolasHug/Surprise.git /tmp/pip-req-build-h9qj8ycd
  Resolved https://github.com/NicolasHug/Surprise.git to commit 2381fb11d0c4bf917cc4b9126f205d0013649966
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [5]:
df = pd.read_csv('data.csv', encoding='ISO-8859-1')
df.dropna(subset=['CustomerID', 'Description'], inplace=True)
df = df[df['Quantity'] > 0]

In [6]:
produk_unik = df[['StockCode', 'Description']].drop_duplicates().reset_index(drop=True)
produk_unik['Description'] = produk_unik['Description'].str.lower()

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(produk_unik['Description'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
indices = pd.Series(produk_unik.index, index=produk_unik['Description']).drop_duplicates()

def get_cbf_recommendations(product_name, top_n=5):
    idx = indices[product_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    product_indices = [i[0] for i in sim_scores]
    return produk_unik['Description'].iloc[product_indices].unique()

In [7]:
cf_data = df.groupby(['CustomerID', 'StockCode'])['Quantity'].sum().reset_index()
cf_data.columns = ['userID', 'itemID', 'rating']

reader = Reader(rating_scale=(cf_data['rating'].min(), cf_data['rating'].max()))
data = Dataset.load_from_df(cf_data[['userID', 'itemID', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
svd = SVD()
svd.fit(trainset)
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 80976.0549


80976.05486796782

In [8]:
def hybrid_recommend(user_id, product_name, alpha=0.5, top_n=5):
    product_ids = df['StockCode'].unique()
    cf_scores = {pid: svd.predict(user_id, pid).est for pid in product_ids}
    try:
        similar_products = get_cbf_recommendations(product_name, top_n=top_n*2)
        cbf_codes = produk_unik[produk_unik['Description'].isin(similar_products)]['StockCode']
        cbf_scores = df[df['StockCode'].isin(cbf_codes)]['StockCode'].value_counts(normalize=True).to_dict()
    except:
        cbf_scores = {}
    hybrid_scores = {}
    for pid in product_ids:
        cf_score = cf_scores.get(pid, 0)
        cbf_score = cbf_scores.get(pid, 0)
        hybrid_scores[pid] = alpha * cf_score + (1 - alpha) * cbf_score
    top_products = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    result = df[df['StockCode'].isin([p[0] for p in top_products])]['Description'].unique()
    return result

In [9]:
user_sample = str(cf_data['userID'].sample(1).values[0])
produk_sample = 'white metal lantern'
hybrid_recommend(user_sample, produk_sample, alpha=0.6, top_n=5)

array(['WHITE METAL LANTERN', 'HANGING METAL HEART LANTERN',
       'HANGING METAL STAR LANTERN', 'LANTERN CREAM GAZEBO ',
       'SET 6 PAPER TABLE LANTERN HEARTS ',
       'WHITE MOROCCAN METAL LANTERN'], dtype=object)

evaluasi tambahan untuk precision@k dan recall@k:

In [12]:
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def precision_recall_at_k(predictions, k=5, threshold=1.0):
    top_n = get_top_n(predictions, n=k)
    precisions = []
    recalls = []
    for uid, user_ratings in top_n.items():
        n_rel = sum((true_r >= threshold) for (_, true_r, _, _, _) in predictions if uid == _)
        n_rec_k = len(user_ratings)
        n_rel_and_rec_k = sum((true_r >= threshold) for (iid, _) in user_ratings
                              for (u, i, true_r, _, _) in predictions if u == uid and i == iid)
        precisions.append(n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0)
        recalls.append(n_rel_and_rec_k / n_rel if n_rel != 0 else 0)
    return np.mean(precisions), np.mean(recalls)

precision, recall = precision_recall_at_k(predictions, k=5)
print(f'Precision@5: {precision:.4f}')
print(f'Recall@5: {recall:.4f}')

Precision@5: 1.0000
Recall@5: 0.0000


berikut evaluasi precision@k dan recall@k versi sederhana untuk mengukur kualitas rekomendasi dari hybrid model:

In [11]:
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def precision_recall_at_k(predictions, k=5, threshold=1.0):
    top_n = get_top_n(predictions, n=k)
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in top_n.items():
        n_rel = sum((true_r >= threshold) for (_, true_r, _, _, _) in filter(lambda x: x[0] in [iid for (iid, _) in user_ratings], predictions))
        n_rec_k = len(user_ratings)
        n_rel_and_rec_k = sum((true_r >= threshold) for (_, true_r, _, _, _) in filter(lambda x: x[0] in [iid for (iid, _) in user_ratings], predictions))
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel else 0
    avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
    avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
    return avg_precision, avg_recall

prec, rec = precision_recall_at_k(predictions, k=5)
print(f"Precision@5: {prec:.4f}")
print(f"Recall@5: {rec:.4f}")

Precision@5: 0.0000
Recall@5: 0.0000


kesimpulan yang bisa saya ambil:
1. model hybrid berhasil dibangun dengan menggabungkan CF (SVD) dan CBF (TF-IDF).
2. valuasi menunjukkan kinerja model dapat diterima (RMSE + Precision/Recall).

referensi:
1. kaggle: [E-Commerce Data](https://www.kaggle.com/datasets/carrie1/ecommerce-data)
2. surprise Library: https://surpriselib.com
3. Ricci, Rokach, Shapira. *Recommender Systems Handbook*. Springer.
4. Aggarwal, C. *Recommender Systems: The Textbook*. Springer.